# Msoup Closure: SPARC Rotation Curves + Lensing Constraints

**Goal**: Test whether ONE set of global Msoup parameters (c_*², ΔM, z_t, w) can simultaneously:

1. **Improve fits** to dwarf/LSB rotation curves (cores & diversity)
2. **Not violate** strong-lensing substructure constraints

This notebook implements the "prove or kill" decision framework.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import sys
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, '..')

from msoup_model import (
    MsoupParams, CosmologyParams, MsoupGrowthSolver,
    RotationCurveFitter, fit_galaxy_sample,
    HaloMassFunction, validate_lcdm_limit
)
from msoup_model.growth import compute_half_mode_mass
from data.sparc import (
    download_sparc, load_rotation_curve,
    get_dwarf_lsb_sample, create_synthetic_sparc_data
)
from data.lensing import (
    LensingConstraints, convert_M_hm_to_wdm_mass,
    subhalo_mass_function_ratio, lensing_signal_prediction
)

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. Load Data

### 1.1 SPARC Rotation Curves

In [ ]:
# Try to download SPARC data
try:
    download_sparc(verbose=True)
    USE_REAL_DATA = True
    print("Using real SPARC data.")
except Exception as e:
    print(f"Could not download SPARC: {e}")
    print("Using synthetic data for demonstration.")
    USE_REAL_DATA = False

In [ ]:
# Load galaxy sample
if USE_REAL_DATA:
    # Get dwarf/LSB galaxies
    try:
        galaxy_names = get_dwarf_lsb_sample(v_max_cut=80, min_points=8)
        galaxies = [load_rotation_curve(name) for name in galaxy_names[:15]]  # Limit for speed
    except Exception as e:
        print(f"Error loading SPARC: {e}")
        galaxies = create_synthetic_sparc_data(n_galaxies=15)
        USE_REAL_DATA = False
else:
    galaxies = create_synthetic_sparc_data(n_galaxies=15)

print(f"\nLoaded {len(galaxies)} galaxies for analysis")
for g in galaxies[:5]:
    print(f"  {g.name}: v_max = {g.v_max:.1f} km/s, {g.n_points} points")

In [ ]:
# Plot example rotation curves
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, gal in enumerate(galaxies[:6]):
    ax = axes[i]
    ax.errorbar(gal.radius, gal.v_obs, yerr=gal.v_err, 
                fmt='o', markersize=4, label='Observed', capsize=2)
    ax.plot(gal.radius, gal.v_baryon(0.5, 0.7), 'g--', label='Baryons')
    ax.plot(gal.radius, gal.v_dm_needed(0.5, 0.7), 'r:', label='DM needed')
    ax.set_xlabel('R [kpc]')
    ax.set_ylabel('V [km/s]')
    ax.set_title(gal.name)
    if i == 0:
        ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig('../results/sparc_sample_curves.png', dpi=150)
plt.show()

### 1.2 Lensing Constraints

In [ ]:
# Load lensing constraints
lensing = LensingConstraints.load_default()
print(lensing.summary())

## 2. Baseline CDM Fits

First, fit all galaxies with standard NFW profiles (no cores).

In [ ]:
cosmo = CosmologyParams()

# CDM baseline (c_*² = 0 means no coring)
params_cdm = MsoupParams(c_star_sq=0, Delta_M=0.5, z_t=2.0, w=0.5)

print("Fitting with CDM (NFW, no cores)...")
results_cdm = fit_galaxy_sample(galaxies, params_cdm, cosmo, verbose=True)

print(f"\nCDM Results:")
print(f"  Total χ² = {results_cdm['total_chi2']:.1f}")
print(f"  Total DOF = {results_cdm['total_dof']}")
print(f"  χ²/DOF = {results_cdm['chi2_per_dof']:.2f}")

## 3. Msoup Parameter Scan

Scan over Msoup parameters to find improvement over CDM while checking lensing constraints.

In [ ]:
# Parameter grid (coarse for speed)
c_star_sq_grid = [0, 50, 100, 200, 400]  # (km/s)²
Delta_M_grid = [0.3, 0.5, 0.8]
z_t_grid = [1.5, 2.0, 3.0]
w_values = [0.5]  # Fix w for now

results_scan = []
n_total = len(c_star_sq_grid) * len(Delta_M_grid) * len(z_t_grid)
print(f"Scanning {n_total} parameter combinations...\n")

i_scan = 0
for c_sq in c_star_sq_grid:
    for DM in Delta_M_grid:
        for zt in z_t_grid:
            i_scan += 1
            if i_scan % 5 == 0:
                print(f"  {i_scan}/{n_total}...")
            
            params = MsoupParams(c_star_sq=c_sq, Delta_M=DM, z_t=zt, w=0.5)
            
            # Fit galaxies
            result = fit_galaxy_sample(galaxies, params, cosmo, verbose=False)
            
            # Compute half-mode mass
            if c_sq > 0:
                solver = MsoupGrowthSolver(params, cosmo, n_k=30, n_z=50)
                solution = solver.solve()
                M_hm = compute_half_mode_mass(solution, z=0)
            else:
                M_hm = 0
            
            # Check lensing
            is_consistent, msg = lensing.is_consistent(M_hm, use_forecasts=False)
            
            results_scan.append({
                'c_star_sq': c_sq,
                'Delta_M': DM,
                'z_t': zt,
                'w': 0.5,
                'chi2': result['total_chi2'],
                'chi2_per_dof': result['chi2_per_dof'],
                'M_hm': M_hm,
                'log_M_hm': np.log10(M_hm) if M_hm > 0 else -np.inf,
                'lensing_ok': is_consistent,
            })

df_scan = pd.DataFrame(results_scan)
print("\nScan complete!")

In [ ]:
# Show best results
print("Top 10 by χ²/DOF (lower is better):")
print(df_scan.sort_values('chi2_per_dof').head(10).to_string(index=False))

In [ ]:
# Filter to lensing-consistent models
df_ok = df_scan[df_scan['lensing_ok']]
print(f"\n{len(df_ok)}/{len(df_scan)} models pass lensing constraints")

if len(df_ok) > 0:
    print("\nBest lensing-consistent models:")
    print(df_ok.sort_values('chi2_per_dof').head(5).to_string(index=False))

## 4. Detailed Fit with Best Parameters

In [ ]:
# Find best lensing-consistent model
if len(df_ok) > 0:
    best = df_ok.sort_values('chi2_per_dof').iloc[0]
else:
    # Fallback to best overall
    best = df_scan.sort_values('chi2_per_dof').iloc[0]
    print("WARNING: No lensing-consistent model found; using best χ²")

print("Best parameters:")
print(f"  c_*² = {best['c_star_sq']} (km/s)²")
print(f"  ΔM = {best['Delta_M']}")
print(f"  z_t = {best['z_t']}")
print(f"  w = {best['w']}")
print(f"\n  χ²/DOF = {best['chi2_per_dof']:.3f}")
print(f"  log10(M_hm) = {best['log_M_hm']:.2f}")
print(f"  Lensing OK: {best['lensing_ok']}")

In [ ]:
# Fit with best parameters
params_best = MsoupParams(
    c_star_sq=best['c_star_sq'],
    Delta_M=best['Delta_M'],
    z_t=best['z_t'],
    w=best['w']
)

results_best = fit_galaxy_sample(galaxies, params_best, cosmo, verbose=True)

# Compare to CDM
Delta_chi2 = results_cdm['total_chi2'] - results_best['total_chi2']
print(f"\n*** Improvement over CDM: Δχ² = {Delta_chi2:.1f} ***")
print(f"    CDM χ²/DOF = {results_cdm['chi2_per_dof']:.3f}")
print(f"    Msoup χ²/DOF = {results_best['chi2_per_dof']:.3f}")

In [ ]:
# Plot individual galaxy fits
fitter_cdm = RotationCurveFitter(cosmo=cosmo, msoup_params=params_cdm)
fitter_msoup = RotationCurveFitter(cosmo=cosmo, msoup_params=params_best)

fig, axes = plt.subplots(3, 3, figsize=(14, 12))
axes = axes.flatten()

for i, gal in enumerate(galaxies[:9]):
    ax = axes[i]
    
    # Data
    ax.errorbar(gal.radius, gal.v_obs, yerr=gal.v_err, 
                fmt='ko', markersize=4, label='Data', capsize=2, alpha=0.7)
    
    # CDM fit
    fit_cdm = fitter_cdm.fit_single_galaxy(
        gal.radius, gal.v_obs, gal.v_err,
        gal.v_gas, gal.v_disk, gal.v_bulge
    )
    ax.plot(gal.radius, fit_cdm['v_model'], 'b--', 
            label=f'CDM (χ²r={fit_cdm["chi2_red"]:.1f})')
    
    # Msoup fit
    fit_msoup = fitter_msoup.fit_single_galaxy(
        gal.radius, gal.v_obs, gal.v_err,
        gal.v_gas, gal.v_disk, gal.v_bulge
    )
    ax.plot(gal.radius, fit_msoup['v_model'], 'r-', linewidth=2,
            label=f'Msoup (χ²r={fit_msoup["chi2_red"]:.1f})')
    
    ax.set_xlabel('R [kpc]')
    ax.set_ylabel('V [km/s]')
    ax.set_title(f"{gal.name} (r_c={fit_msoup['r_c']:.1f} kpc)")
    ax.legend(fontsize=7, loc='lower right')

plt.tight_layout()
plt.savefig('../results/rotation_curve_fits.png', dpi=150)
plt.show()

## 5. Lensing Constraint Check

Does the best-fit M_hm violate lensing constraints?

In [ ]:
# Compute M_hm for best model
if best['c_star_sq'] > 0:
    solver_best = MsoupGrowthSolver(params_best, cosmo, n_k=50, n_z=80)
    solution_best = solver_best.solve()
    M_hm_best = compute_half_mode_mass(solution_best, z=0)
else:
    M_hm_best = 0

print(f"Best-fit half-mode mass: M_hm = {M_hm_best:.2e} M_sun")
print(f"                         log10(M_hm) = {np.log10(M_hm_best) if M_hm_best > 0 else -np.inf:.2f}")

# Check against all constraints
is_ok, msg = lensing.is_consistent(M_hm_best, use_forecasts=False)
print(f"\nConsistent with HARD constraints: {is_ok}")
print(msg)

# Also check forecasts
is_ok_forecast, msg_forecast = lensing.is_consistent(M_hm_best, use_forecasts=True)
print(f"\nConsistent with FORECAST constraints: {is_ok_forecast}")

In [ ]:
# Visualize constraint space
fig, ax = plt.subplots(figsize=(10, 6))

# Plot scan results
scatter = ax.scatter(
    df_scan['log_M_hm'].replace(-np.inf, -1),
    df_scan['chi2_per_dof'],
    c=df_scan['c_star_sq'],
    cmap='viridis',
    s=80,
    alpha=0.7,
    edgecolor='k'
)
plt.colorbar(scatter, label=r'$c_*^2$ [(km/s)$^2$]')

# Mark CDM baseline
ax.axhline(results_cdm['chi2_per_dof'], color='b', linestyle='--', 
           label=f'CDM baseline (χ²/DOF = {results_cdm["chi2_per_dof"]:.2f})')

# Mark lensing constraints
for c in lensing.get_hard_constraints():
    if c.is_upper_limit:
        ax.axvline(np.log10(c.M_hm_limit), color='r', linestyle=':', alpha=0.5)
        ax.text(np.log10(c.M_hm_limit), ax.get_ylim()[1]*0.95, 
                c.name, fontsize=8, rotation=90, va='top')

ax.axvline(np.log10(M_hm_best) if M_hm_best > 0 else -1, 
           color='green', linewidth=2, label='Best fit')

ax.set_xlabel(r'$\log_{10}(M_{hm} / M_\odot)$')
ax.set_ylabel(r'$\chi^2$/DOF')
ax.set_title('SPARC Fit Quality vs Half-Mode Mass')
ax.legend(loc='upper left')
ax.set_xlim(-1, 12)

plt.tight_layout()
plt.savefig('../results/constraint_space.png', dpi=150)
plt.show()

## 6. Subhalo Mass Function Comparison

In [ ]:
# Predicted subhalo MF ratio
M_sub = np.logspace(6, 10, 100)

fig, ax = plt.subplots(figsize=(10, 6))

# Best-fit model
if M_hm_best > 0:
    ratio_best = subhalo_mass_function_ratio(M_sub, M_hm_best)
    ax.semilogx(M_sub, ratio_best, 'r-', linewidth=2, 
                label=f'Best fit (M_hm = {M_hm_best:.1e})')

# CDM
ax.axhline(1, color='b', linestyle='--', label='CDM')

# Lensing sensitivity region
ax.axvspan(1e6, 1e9, alpha=0.2, color='orange', label='Lensing sensitive')

# Constraint limits
for c in lensing.get_hard_constraints()[:2]:
    if c.is_upper_limit:
        ratio_limit = subhalo_mass_function_ratio(M_sub, c.M_hm_limit)
        ax.semilogx(M_sub, ratio_limit, ':', alpha=0.5, 
                    label=f'{c.name} limit')

ax.set_xlabel(r'Subhalo mass $M_{sub}$ [$M_\odot$]')
ax.set_ylabel('SHMF / SHMF$_{CDM}$')
ax.set_title('Subhalo Mass Function Suppression')
ax.legend()
ax.set_ylim(0, 1.2)

plt.tight_layout()
plt.savefig('../results/subhalo_mf.png', dpi=150)
plt.show()

## 7. "Prove or Kill" Decision

Evaluate the three potential falsifiers.

In [ ]:
print("="*70)
print("MSOUP CLOSURE MODEL: PROVE OR KILL EVALUATION")
print("="*70)

# Falsifier 1: Wrong redshift turn-on
print("\n1. REDSHIFT TURN-ON CHECK")
print(f"   Best-fit z_t = {best['z_t']}")
if best['z_t'] > 5:
    print("   STATUS: CONCERN - Effect required too early")
    falsifier_1 = True
else:
    print("   STATUS: OK - Turn-on at reasonable epoch")
    falsifier_1 = False

# Falsifier 2: Wrong scale dependence
print("\n2. SCALE DEPENDENCE CHECK")
if M_hm_best > 0:
    k_hm = np.pi / (M_hm_best / (cosmo.rho_m_0))**(1/3)
    print(f"   Half-mode scale: k_hm ~ {k_hm:.2f} h/Mpc")
    print(f"   Half-mode mass: M_hm = {M_hm_best:.2e} M_sun")
    if k_hm < 0.1 or k_hm > 100:
        print("   STATUS: CONCERN - Suppression scale inconsistent")
        falsifier_2 = True
    else:
        print("   STATUS: OK - Suppression at expected scales")
        falsifier_2 = False
else:
    print("   No suppression (CDM-like)")
    falsifier_2 = False

# Falsifier 3: Multi-probe inconsistency
print("\n3. MULTI-PROBE CONSISTENCY CHECK")
print(f"   SPARC χ²/DOF: {results_best['chi2_per_dof']:.3f}")
print(f"   CDM χ²/DOF: {results_cdm['chi2_per_dof']:.3f}")
print(f"   Improvement: Δχ² = {Delta_chi2:.1f}")

is_ok, _ = lensing.is_consistent(M_hm_best, use_forecasts=False)
print(f"   Lensing consistent: {is_ok}")

if Delta_chi2 > 0 and is_ok:
    print("   STATUS: OK - SPARC improved, lensing consistent")
    falsifier_3 = False
elif Delta_chi2 <= 0:
    print("   STATUS: NEUTRAL - No improvement over CDM")
    falsifier_3 = False
else:
    print("   STATUS: CONCERN - Cannot satisfy both probes")
    falsifier_3 = True

# Final verdict
print("\n" + "="*70)
print("VERDICT")
print("="*70)

if falsifier_1 or falsifier_2 or falsifier_3:
    print("\n*** MODEL SHOWS TENSION ***")
    if falsifier_1:
        print("  - Falsifier 1: Wrong redshift turn-on")
    if falsifier_2:
        print("  - Falsifier 2: Wrong scale dependence")
    if falsifier_3:
        print("  - Falsifier 3: Multi-probe inconsistency")
else:
    print("\n*** MODEL SURVIVES INITIAL TESTS ***")
    print("  No falsifiers triggered.")
    print(f"  Best parameters: c_*²={best['c_star_sq']}, ΔM={best['Delta_M']}, z_t={best['z_t']}")

## 8. Save Results

In [ ]:
# Save results as JSON
results_summary = {
    'model': 'Msoup closure',
    'version': '0.1.0',
    'best_params': {
        'c_star_sq': float(best['c_star_sq']),
        'Delta_M': float(best['Delta_M']),
        'z_t': float(best['z_t']),
        'w': float(best['w']),
    },
    'sparc_results': {
        'n_galaxies': len(galaxies),
        'chi2_per_dof_cdm': float(results_cdm['chi2_per_dof']),
        'chi2_per_dof_msoup': float(results_best['chi2_per_dof']),
        'delta_chi2': float(Delta_chi2),
    },
    'lensing_results': {
        'M_hm': float(M_hm_best) if M_hm_best > 0 else None,
        'log_M_hm': float(np.log10(M_hm_best)) if M_hm_best > 0 else None,
        'consistent_hard': is_ok,
        'consistent_forecast': is_ok_forecast,
    },
    'falsifiers': {
        'wrong_redshift_turnon': falsifier_1,
        'wrong_scale_dependence': falsifier_2,
        'multiprobe_inconsistent': falsifier_3,
    },
    'verdict': 'survives' if not (falsifier_1 or falsifier_2 or falsifier_3) else 'tension',
    'data_used': {
        'sparc': 'real' if USE_REAL_DATA else 'synthetic',
        'lensing': 'compiled constraints from Gilman+2020, Vegetti+2018, Hsueh+2020, Enzi+2021',
    },
}

with open('../results/fit_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("Results saved to results/fit_results.json")

In [ ]:
# Save scan results
df_scan.to_csv('../results/parameter_scan.csv', index=False)
print("Scan results saved to results/parameter_scan.csv")